# Ánalisis de datos de Electroencefalograma

## Secciones del código
1. El primer recuadro lo que hace es cargar las funciones necesarias para el correcto funcionamiento del cuaderno y cargar el módulo donde están todas las funciones necesarias
1. En el segundo recuadro pide la dirección del archivo, crea una carpeta para almacenar los resultados del análisis y obtiene las variables y datos ya proporcionados por el documento que se está analizando
1. El tercer recuadro es el más complejo ya que ahí comienza el análisis y obtiene muchos resultados, entre ellos:
    1. Array con los datos del voltaje en cada canal en cada punto temporal del registro
    1. Resultado de realizar el _Current Source Density Analysis_
    1. Ubicaciones de los centros de masa en cada punto del tiempo
    1. Descripción de las trayectorias de los centros de masa ( En coordenadas cartesianas y polares )
1. El cuarto recuadro realiza una conversión de los datos a otro formato necesario para la graficación posterior y da distintos parametros para las mismas
1. En el quinto recuadro genera una imagen del registro en todo el tiempo que dura el archivo
1. La sexta sección genera Gif's donde muestra el _CSDA_ realizado y las trayectorias de los centros de masa
1. En la última parte será la visualización gráfica de todos los resultados obtenidos y visualizables, entre ellos:
    1. Topograma de la actividad de corriente en toda la región
    1. Registro del voltaje registrado en cada canal
    1. Trayectorias de los centros de masa en coordenadas polares
    1. _CSDA_ registrado en cada canal con formato de array
> En la última gráfica poseé cada sección un temporizador que marca el tiempo desde que se inició el experimento

In [1]:
import JLD.load
import JLD.save
import DelimitedFiles.readdlm
using Plots
using Suppressor
push!(LOAD_PATH, ".")
using Funciones_EEG

[ Info: Precompiling Funciones_EEG [top-level]


In [7]:
#-------------------------------------------------------------------------------------------------------------#
# Aquí se da la dirección donde está el archivo a trabajar
Direccion = "/home/angel/Documents/Datos/Puebla_Records/derek_20_jun_2023/derek2_230620000d_open.dat";
Dir = split( Direccion,".")[1];
Archivo = split(basename(Direccion),".")[1];
Datos = readdlm( Direccion );
ruta_resultados = joinpath(Dir * "_Resultados" );
mkpath( ruta_resultados );
#-------------------------------------------------------------------------------------------------------------#
Vars = VarsDeDoc(Datos);
NomVar = Archivo*"_Vars.jld";
ruta_archivo = joinpath( ruta_resultados , NomVar );
save( ruta_archivo , Vars )
Headers , Canales = DatosDeTodos( Datos , 9 , 15 );

In [8]:
#-------------------------------------------------------------------------------------------------------------#
Coordenadas , lados , Prohibidas = Coordenadas_Y_Lados( OrdenMaximo , Headers );
elementos = [ -1 0 1 ]
combs = COMS(elementos);
#-------------------------------------------------------------------------------------------------------------#
MVCuadComp = ArrayParaHeatmap( lados , Canales , Prohibidas , Coordenadas );
NomCuad = Archivo*"_DatosEnCuadrado.jld"
ruta_archivo = joinpath( ruta_resultados , NomCuad )
save( ruta_archivo , "MVCuacComp" , MVCuadComp )
#-------------------------------------------------------------------------------------------------------------#
MVYPromedios = PromVacios( Prohibidas , Coordenadas , combs , Headers , Canales , MVCuadComp );
MVcsda = CSDAZapfe( MVYPromedios );
NomCSDA = Archivo*"_CSDA.jld";
ruta_archivo = joinpath( ruta_resultados , NomCSDA );
MVcsdaListo = EliminaProhibidas( MVcsda , Prohibidas , Coordenadas );
save( ruta_archivo , "MVcsda" , MVcsda )
#-------------------------------------------------------------------------------------------------------------#
CMP , CMN = CentrosDeMasa( MVcsdaListo , 0.5 );
NomCM = Archivo*"_CM.jld";
ruta_archivo = joinpath( ruta_resultados , NomCM );
save( ruta_archivo , "CMP" , CMP , "CMN" , CMN )
#-------------------------------------------------------------------------------------------------------------#
( DistTol , TTol , Tmin ) = ( 1.5 , 3 , 3 )
TraysPos = Dinamica_LV( CMP , DistTol , TTol , Tmin );
TraysNeg = Dinamica_LV( CMN , DistTol , TTol , Tmin );
NomTrays = Archivo*"_Trayectorias.jld";
ruta_archivo = joinpath( ruta_resultados , NomTrays );
save( ruta_archivo , "TP" , TraysPos , "TN" , TraysNeg )
#-------------------------------------------------------------------------------------------------------------#
( IniciosP , FinalesP ) = IniciosYFinales( TraysPos );
( IniciosN , FinalesN ) = IniciosYFinales( TraysNeg );
TraysPos = RellenaHuecos( TraysPos , IniciosP , FinalesP );
TraysNeg = RellenaHuecos( TraysNeg , IniciosN , FinalesN );
TraysPosP = ArreglaCoordenadas( TraysPos , lados );
TraysNegP = ArreglaCoordenadas( TraysNeg , lados );
NomTPs = Archivo*"_TraysPolares.jld"
ruta_archivo = joinpath( ruta_resultados , NomTPs )
save( ruta_archivo , "TPP" , TraysPosP , "TNP" , TraysNegP )

In [9]:
pesos , CT , CoorCSDA = DatosParaTopograma( Coordenadas , Prohibidas , MVcsdaListo , lados );
CTPs = [ :crimson , :brown3 , :red , :indianred3 , :coral3 , :indianred2 , :orangered4 , :red4 ];
CTNs = [ :aqua , :cyan3 , :turquoise , :deepskyblue , :skyblue3 , :teal , :blue4 , :midnightblue ];
( extra , PC , nd , longtray , fr ) = ( 1 , 15 , 2 , 5 , Vars[ "Rate" ] )
# "extra"     Espacio extra para los máximos en las gráficas
# "PC"        Cantidad de cachos propuesta 
# "nd"        Numero de digitos del maximo de cachos
# "longtray"  Longitud de trayectoria en frame 
# "fr"        Frecuencia de muestreo
( cachos , Bines ) = DivideAnimacion( PC , pesos );

In [10]:
@suppress begin
    Registro = Registro_Completo( Headers , Canales , ruta_resultados )
end

In [11]:
@suppress begin
    TodosGIFS = Animaciones_Trays( extra , cachos , nd , longtray , fr , MVcsda , ruta_resultados , 
                                 IniciosP , FinalesP , TraysPosP , CT , IniciosN , FinalesN ,
                                 TraysNegP , CTPs , CTNs , Bines , nothing , nothing , nothing )
end

"El proceso terminó"

In [7]:
#Se recomienda que solo se haga un bin a la vez ya que es muy pesado
@suppress begin
    Todos = Animaciones_Trays( extra , cachos , nd , longtray , fr , MVcsda , ruta_resultados , IniciosP , 
                            FinalesP , TraysPosP , CT, IniciosN , FinalesN , TraysNegP , CTPs, CTNs , Bines, 
                            Headers, Canales , pesos , CoorCSDA )
end

"El proceso terminó"